# A/B/n Testing Course

## 0.1 Imposts

In [39]:
import pandas as pd
import numpy as np
import math
from statsmodels.stats import api as sms
from statsmodels.stats.gof import chisquare_effectsize
from statsmodels.stats.power import GofChisquarePower,TTestIndPower
from scipy.stats import chi2_contingency
from itertools import combinations
from statsmodels.sandbox.stats.multicomp import multipletests

# 1.0 Load Data

In [11]:
d = {
  'variant': ['interect','connect','learn','help','sevices'],
  'visits': [10283,2742,2747,3180,2064],
  'clicks_all': [3714,1587,1652,1717,1348],
  'clicks_link': [42,53,21,38,45]
}

#controle: interect

data = pd.DataFrame(d)
data['conversion'] = data['clicks_link'] / data['clicks_all']
data

,variant,visits,clicks_all,clicks_link,conversion
0,interect,10283,3714,42,0.011309
1,connect,2742,1587,53,0.033396
2,learn,2747,1652,21,0.012712
3,help,3180,1717,38,0.022132
4,sevices,2064,1348,45,0.033383


# 2.0 Design de Experimentos

## 2.1 Formação das Hipóteses

H0: Não há nenhuma diferença entre o CTR das variantes da página<br>
H1: Há diferença entre os CTR das variantes da página

Teste Bi-Caudal

## 2.2 Parâmetros do experimentos

In [12]:
#nível de confiança
confidence_level = 0.95

#nível de significancia
significance_level = 1 - confidence_level

#poder estatístico
power = 0.8

k = len(data['clicks_all'])

actual_dist = data['clicks_link'] / data['clicks_link'].sum()
expected_dist = [1/k] * k #distribuição se todas as páginas fossem iguais, distribuição igualitária
effect_size = chisquare_effectsize(expected_dist,actual_dist) #Efeito que espero observar na configuração que eu tenho
effect_size

0.2665685871331627

Chi-Squared: Teste de inferência estatística para detectar independência ou ajuste entre duas variáveis categóricas:
  - Chi Square: Good of Fitness -> Inferir a distância entre a distribuição da amostra e a distribuição da população
  - Chi Square: Teste de independência -> Inferir se existe uma relação ou dependência entre 2 variáveis categóricas

In [14]:
#sample size - Tamanho da amostra
sample_n = TTestIndPower().solve_power(
  effect_size=effect_size,
  power=power,
  alpha=significance_level
)

sample_n = math.ceil(sample_n)
print(f'O tamanho da amostra de cada grupo controle é de {sample_n}.')
print(f'O tamanho total da amostra é de {sample_n*5}.')

O tamanho da amostra de cada grupo controle é de 222.
O tamanho total da amostra é de 1110.


# 3.0 Aplicação do Teste de Chi Squared

In [18]:
data['no_clicks_link'] = data['clicks_all'] - data['clicks_link']
df = data[['variant','clicks_link','no_clicks_link']]
df = df.set_index('variant')
df

,clicks_link,no_clicks_link
variant,,
interect,42,3672
connect,53,1534
learn,21,1631
help,38,1679
sevices,45,1303


In [28]:
chi_val,pval,dof,expected = chi2_contingency(df)
print(f'p-value:{pval}')

if pval < significance_level:
  print(f'Rejeita a hipótese nula, existe uma diferença entre as variantes')
else:
  print(f'Falha em rejeitar a hipótese nula, NÃO existe uma diferença entre as variantes')

p-value:2.0959498129984567e-09
Rejeita a hipótese nula, existe uma diferença entre as variantes


# 4.0 Post-hoc Testing

In [54]:
all_combinations = list(combinations(df.index,2)) #criando uma lista de para par
p_values = []
for comb in all_combinations:
  new_df = df[(df.index == comb[0]) | (df.index == comb[1])]
  chi_val,pval,dof,expected = chi2_contingency(new_df)
  p_values.append(pval)

# Correction of Bonferroni
reject_list,correct_p_values = multipletests(p_values,method='bonferroni')[:2]

In [55]:
for comb, p_val, corr_p_val, reject in zip(all_combinations,p_values,correct_p_values,reject_list):
  print(f'\n{comb}: p_value: {p_val}, correted_p_value: {corr_p_val}, Reject: {reject}')


('interect', 'connect'): p_value: 5.3676772349808135e-08, correted_p_value: 5.367677234980813e-07, Reject: True

('interect', 'learn'): p_value: 0.7616980743361713, correted_p_value: 0.8463311937068569, Reject: False

('interect', 'help'): p_value: 0.0031030587017400212, correted_p_value: 0.0062061174034800425, Reject: True

('interect', 'sevices'): p_value: 1.798089447385411e-07, correted_p_value: 8.990447236927055e-07, Reject: True

('connect', 'learn'): p_value: 0.00013292868361715983, correted_p_value: 0.0004430956120571994, Reject: True

('connect', 'help'): p_value: 0.06144184057612575, correted_p_value: 0.08777405796589394, Reject: False

('connect', 'sevices'): p_value: 1.0, correted_p_value: 1.0, Reject: False

('learn', 'help'): p_value: 0.0508958228881819, correted_p_value: 0.08482637148030317, Reject: False

('learn', 'sevices'): p_value: 0.00020374035733741825, correted_p_value: 0.0005093508933435456, Reject: True

('help', 'sevices'): p_value: 0.07301998638337415, corret

'interect', 'connect' -> Há uma relação, diferença -> p_value = 5.367677234980813e-07 <br>
'interect', 'sevices' -> Há uma relação, diferença -> p_value = 1.7980894473854111e-06 <br>
'interect', 'help' -> Há uma relação, diferença -> p_value = 0.03103058701740021 <br>

'connect', 'help' -> Não há uma relação, diferença <br>
'connect', 'sevices' -> Não há uma relação, diferença <br>
'help', 'sevices' -> Não há uma relação, diferença <br>
Não há diferença entre essas variáveis

In [ ]:
# One Sample T-Test 
# Two Sample T-Test 

# One Sample Z-Test 
# Two Sample Z-Test 

# Chi Squared: Goodness of Fit 
# Chi Squared: Independence Test 

# One Sample ANOVA 
# Two Sample ANOVA 

# 2 variantes = T Test (n > 30)
# 2 variantes = Z Test (n < 30)

# categorica x caterorica = ANOVA 
# catetogica x numerica = T-Test 

# 3 ou mais variantes = ANOVA